from google.colab import drive
drive.mount('/content/drive')

In [0]:
import os
base_dir = '/content/drive/My Drive/Colab/'
train_dir = os.path.join(base_dir,'train')
validation_dir = os.path.join(base_dir,'validation')

# Directory with our training safe pictures
train_cancer_dir = os.path.join(train_dir,'cancer')

# Directory with our training safe pictures
train_safe_dir = os.path.join(train_dir,'safe')

# Directory with our validation safe pictures
validation_cancer_dir = os.path.join(validation_dir,'cancer')

# Directory with our validation safe pictures
validation_safe_dir = os.path.join(validation_dir,'safe')


train_cancer_fnames = os.listdir(train_cancer_dir)
train_safe_fnames = os.listdir(train_safe_dir)

In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [0]:
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from tensorflow.keras.preprocessing.image import array_to_img, img_to_array, load_img

img_path = os.path.join(train_cancer_dir, train_cancer_fnames[2])
img = load_img(img_path, target_size=(512, 512))  # this is a TIFF image, we are neglating the transperent channel
x = img_to_array(img)  # Numpy array with shape (512, 512, 3)
x = x.reshape((1,) + x.shape)  # Numpy array with shape (1, 512, 512, 3)


In [5]:
# Adding rescale, rotation_range, width_shift_range, height_shift_range,
# shear_range, zoom_range, and horizontal flip to our ImageDataGenerator
train_datagen = ImageDataGenerator(
    rescale=1./255)

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator(rescale=1./255)

# Flow training images in batches of 2 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        train_dir,  # This is the source directory for training images
        target_size=(512, 512),  
        batch_size=2,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

# Flow validation images in batches of 2 using test_datagen generator
validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(512, 512),
        batch_size=2,
        class_mode='binary')

/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image.py:1728: UserWarning: Using '.tiff' files with multiple bands will cause distortion. Please verify your output.
  warnings.warn('Using \'.tiff\' files with multiple bands '


Found 78 images belonging to 2 classes.
Found 26 images belonging to 2 classes.


In [0]:
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.optimizers import RMSprop

# Our input feature map is 512x512x3: 512x512 for the image pixels, and 3 for
# the three color channels: R, G, and B
img_input = layers.Input(shape=(512, 512, 3))

# First convolution extracts 16 filters that are 3x3
# Convolution is followed by max-pooling layer with a 2x2 window
x = layers.Conv2D(16, 3, activation='relu')(img_input)
x = layers.MaxPooling2D(2)(x)

# Second convolution extracts 32 filters that are 3x3
# Convolution is followed by max-pooling layer with a 2x2 window
x = layers.Conv2D(32, 3, activation='relu')(x)
x = layers.MaxPooling2D(2)(x)

# Third convolution extracts 64 filters that are 3x3
# Convolution is followed by max-pooling layer with a 2x2 window
x = layers.Convolution2D(64, 3, activation='relu')(x)
x = layers.MaxPooling2D(2)(x)

# Flatten feature map to a 1-dim tensor
x = layers.Flatten()(x)

# Create a fully connected layer with ReLU activation and 512 hidden units
x = layers.Dense(512, activation='relu')(x)

# Add a dropout rate of 0.5
x = layers.Dropout(0.5)(x)

# Create output layer with a single node and sigmoid activation
output = layers.Dense(1, activation='sigmoid')(x)

# Configure and compile the model
model = Model(img_input, output)
model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=0.0005),
              metrics=['acc'])

In [7]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch=39,
      epochs=10,
      validation_data=validation_generator,
      validation_steps=13,
      verbose=2)

Epoch 1/10
 - 93s - loss: 5.9874 - acc: 0.4615 - val_loss: 0.5705 - val_acc: 0.8462
Epoch 2/10
 - 81s - loss: 0.3329 - acc: 0.8974 - val_loss: 0.1412 - val_acc: 0.9615
Epoch 3/10


KeyboardInterrupt: ignored

In [0]:
from tensorflow.keras.preprocessing.image import array_to_img, img_to_array, load_img
img_pathh=base_dir+'train/cancer/3.tiff'
img = load_img(img_pathh, target_size=(512, 512))  # this is a TIFF image, we are neglating the transperent channel
x = img_to_array(img)  # Numpy array with shape (512, 512, 3)
x = x.reshape((1,) + x.shape)

In [0]:
result=model.predict(x)

In [10]:
result.shape

(1, 1)

In [0]:

if result[0][0]==1:
    prediction='safe'
else:
    prediction='cancer'

In [12]:
prediction

'cancer'

In [0]:

from keras.models import load_model

model.save('/content/drive/My Drive/Colab/pranab1.h5')